[View in Colaboratory](https://colab.research.google.com/github/mitreilab/autonomousfuture/blob/master/reilAutomation.ipynb)

## Import OES Data

In [0]:
import pandas
oes = pandas.read_csv("https://raw.githubusercontent.com/mitreilab/autonomousfuture/master/oes/realestate_oes_nat4d_2007-2016.csv", low_memory = False)

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

onet.sqlite


## Import ONet Data

In [0]:
import sqlite3
onetConnection = sqlite3.connect('onet.sqlite')
onetConnection.